In [5]:
!conda

usage: conda [-h] [-V] command ...

conda is a tool for managing and deploying applications, environments and packages.

Options:

positional arguments:
  command
    clean             Remove unused packages and caches.
    compare           Compare packages between conda environments.
    config            Modify configuration values in .condarc. This is modeled
                      after the git config command. Writes to the user
                      .condarc file (/Users/cas/.condarc) by default. Use the
                      --show-sources flag to display all identified
                      configuration locations on your computer.
    create            Create a new conda environment from a list of specified
                      packages.
    info              Display information about current conda install.
    init              Initialize conda for shell interaction.
    install           Installs a list of packages into a specified conda
                      environment.
  